In [1]:
import os

In [6]:
%pwd

'c:\\Users\\mehak\\OneDrive\\NLP_Project\\Text-Summarizer---NLP'

In [5]:
os.chdir("../")

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True) 
class ModelEvaluationConfig:  
    root_dir: Path
    data_path: Path
    tokenizer_path: Path
    model_path: Path
    metric_file_name: Path

In [8]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml
from textSummarizer.utils.common import create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:

        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_path = config.tokenizer_path,
            model_path = config.model_path,
            metric_file_name = config.metric_file_name
        )

        return model_evaluation_config

In [13]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch
import pandas as pd
from tqdm import tqdm

In [14]:
import evaluate

In [24]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def generate_batch_sized_chunks(self,list_of_elements, batch_size):
    
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]
    
    def calculate_metric_on_test_ds(self,dataset, metric, model, tokenizer, 
                               batch_size=16, 
                               column_text="article", 
                               column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):
            
            inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                            padding="max_length", return_tensors="pt")
            
            summaries = model.generate(input_ids=inputs["input_ids"],
                            attention_mask=inputs["attention_mask"], 
                            length_penalty=0.8, num_beams=8, max_length=128)
            
            ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''
            
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                    clean_up_tokenization_spaces=True)  
                for s in summaries]      
            
            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
            
            metric.add_batch(predictions=decoded_summaries, references=target_batch)
            
        score = metric.compute()
        return score
    
    def evaluate(self):
        
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path)
       
        dataset = load_from_disk(self.config.data_path)


        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
  
        rouge_metric = evaluate.load('rouge')

        score = self.calculate_metric_on_test_ds(
            dataset['test'][0:10], rouge_metric, model, tokenizer, 
            batch_size=2, column_text='dialogue', column_summary='summary'
        )

        #print(score)

        rouge_dict = {rn: score[rn] for rn in rouge_names}

        df = pd.DataFrame(rouge_dict, index = ['pegasus'] )
        df.to_csv(self.config.metric_file_name, index=False)

In [25]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.evaluate()
except Exception as e:
    raise e

[2025-02-11 00:48:44,364: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-11 00:48:44,377: INFO: common: yaml file: params.yaml loaded successfully]


[2025-02-11 00:48:44,380: INFO: common: created directory at: artifacts]
[2025-02-11 00:48:44,380: INFO: common: created directory at: artifacts/model_evaluation]


100%|██████████| 5/5 [02:47<00:00, 33.52s/it]

[2025-02-11 00:51:39,146: INFO: rouge_scorer: Using default tokenizer.]


In [26]:
#gen_kwargs = {"length_penalty": 0.8, "num_beams": 8, "max_length": 128}

In [37]:
#dataset = load_from_disk("C:/Users/mehak/OneDrive/NLP_Project/Text-Summarizer---NLP/artifacts/data_transformation/transformed_dataset")
#sample_text = dataset["test"][9]["dialogue"]
#reference = dataset["test"][9]["summary"]

In [29]:
#tokenizer = AutoTokenizer.from_pretrained("C:/Users/mehak/OneDrive/NLP_Project/Text-Summarizer---NLP/artifacts/model_trainer/tokenizer")
#model = AutoModelForSeq2SeqLM.from_pretrained("C:/Users/mehak/OneDrive/NLP_Project/Text-Summarizer---NLP/artifacts/model_trainer/pegasus-samsum-model")

In [34]:
#from transformers import pipeline
#pipe = pipeline("summarization", model=model, tokenizer=tokenizer)

In [39]:
#print("Dialogue:")
#print(sample_text)

#print("\nReference Summary:")
#print(reference)

#print("\nModel Summary:")
#print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])